In [303]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json
import time
from requests.exceptions import ConnectionError

https://stackoverflow.com/questions/62335537/scraping-text-from-kickstarter-projects-return-nothing

In [173]:
base = "/Users/ivankoh/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_74056/4110031066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [ ]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

# get the slug from the urls. the slug is the input to the GraphQL API call 
slugs = list(map(get_slug, df_1['main_url'].tolist()))
# while iterating through all projects, add the story and risk to these lists. At the end len of both list 
# should be the same as the len of the dataframe. if all good then add as columns to the dataframe df. 
story_list = []
risk_list = []

In [299]:
block_size = len(df_1) // 100
num_blocks = len(df_1) // 2035
num_blocks
2 * block_size
print(0 * block_size, 1 * block_size)
print(len(slugs))

0 2035
203510


In [349]:
# instantiate a new Session 
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
           'cookie': '_ksr_session=fc2U7qXXaRN91foNiE53NyU3s181NZO0Ll57xPkYxZ5iyUNgus35a0HwsPBTfViBY%2ByAKbtpRirAVLxOGKzG%2BYMOmsLRBPujZep%2Fca%2B1%2FXzW3xX56VXkh5w6ItYhIctEFifQQhw3rTmvoljyHw%3D%3D--4pK6xBEgChjqgmte--LH4Q1qSnhU%2FYX9JgTzGuSQ%3D%3D;'
          }
# GET request to kickstarter to get some token needed for later 
# r = s.get("https://www.kickstarter.com", headers=headers)
# BeautifulSoup parses the html returned as a response from the GET request 
# soup = BeautifulSoup(r.text, 'html.parser')
# xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

# PARAMS
counter = 0
block_number = 0

# pause for 5 seconds every 5 iterations
for i in range(2895, len(slugs)):
    slug = slugs[i]
    if counter == 5:
        print("TAKING A NAP ZZZ...")
        time.sleep(20)
        counter = 0
    counter += 1
    print(f"--------{slug}------")
    # POST request to GraphQL API situated in kickstarter domain 
    # response from POST request should contain the story and risks
    connect_try = 0
    not_found = True
    while connect_try < 30 and not_found:       
        try:
            r = s.post("https://www.kickstarter.com/graph",
                headers= {
                      'content-type': 'application/json',
                      'x-csrf-token': 'GgpaCc5tY-2VRVj1bE1aLOPKLkwRFEbIF205o2WorHLvaUEoOIPiZJSx4rbQg0csVbwBfq-jKGXPw2j42dvQCQ',
                      'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
                      'cookie': '_ksr_session=aVeEJvdlSYzJIE8X7f3v85C2T7b9Q30EN0ONWdeWjMLei6H06q6eRA3DZfCsbal40AfBTSbfczmlTv2ZIpvdyykn4QQYZdSlIzlalRivb6Xlb%2BXhXDhBNJg7KPYEEyFFIX3Qz8%2F61YNIoV136X5ju%2BE7jrqQFsGEB1%2FhAwJZ7kjD6QiwTDFdWWmJDr2wmREspLGXKUO0gicnsrUxQJPdTQMCcFNXy6sx95x5S%2FiNs4xVq0t4jWv%2BaiSnwBp8pC6Xu81FBhpD%2F9sNLZ9YyS5Obzbv0jI%3D--tM1UdSPo8XKxtj%2Bt--WQhnQX4EovqKwzBU9tiWIg%3D%3D'
                },
                json = {
                    "operationName":"Campaign",
                    "variables":{
                        "slug": slug
                    },
                    "query": query
                })
            not_found = False
        except ConnectionError:
            print("Connection Error")
            time.sleep(5)
            connect_try += 1
    try:
        result = r.json()
        print("-------STORY--------")
        story_html = result["data"]["project"]["story"]
        soup = BeautifulSoup(story_html, 'html.parser')
        curr_story = []
        for i in soup.find_all('p'):
            print(i.get_text())
            curr_story.append(i.get_text())
        story_list2.append(" ".join(curr_story) if curr_story else "")

        print("-------RISKS--------")
        print(result["data"]["project"]["risks"])
        risk_list2.append(result["data"]["project"]["risks"] if result["data"]["project"]["risks"] else "")
    except:
        print(r)
        story_list2.append("")
        risk_list2.append("")
        continue

--------justinmcroberts/justins-18th-musical-project------
<Response [429]>
--------slydersmith/slyder-smith-and-the-oblivion-kids-debut-album------
<Response [429]>
--------frenchyandthepunk/zen-ghost-the-new-album-by-frenchy-and-the-punk------
<Response [429]>
--------flavors/blanketpals-tm------
<Response [429]>
--------108366845/kickstarter-onboarding-teardown-supreme------
<Response [429]>
TAKING A NAP ZZZ...


KeyboardInterrupt: 

## Latest Iteration Ran Till: 4289, Local copy latest: 0_4289

In [348]:
#SAVE LIST IN LOCAL FOR USE LATER. NAME FILE AS {from}_{to]_{story/risk}.json
story_file_name = "0_4289_story" #CHANGE THIS
risk_file_name = "0_4289_risk" #CHANGE THIS 

story_file_name += ".json"
risk_file_name += ".json"

with open(story_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(story_list2, f, indent=2) 

with open(risk_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(risk_list2, f, indent=2) 
    
# with open(story_file_name, 'r') as f:
#     story_subset = json.load(f)

In [347]:
print("risk:", len(risk_list2),"\n", "story:", len(story_list2))

risk: 4290 
 story: 4290


In [350]:
print("LATEST CALL DF ROW", "\n")
print(df_1.iloc[len(story_list2)-1]['blurb'])

print("LATEST STORY IN LIST", "\n")
print(story_list2[-1])

print("LATEST RISK IN LIST", "\n")
print(risk_list2[-1])

LATEST CALL DF ROW 

Herbal tea time under a pergola
LATEST STORY IN LIST 


LATEST RISK IN LIST 




In [360]:
# story_list2 = story_list[:-130]
# risk_list2 = risk_list[:-130]
story_list2 = story_list2[:-5]
risk_list2 = risk_list2[:-5]

In [165]:
s = requests.Session()

In [234]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": "leightonconnor/akashic-titan-blue-bolt"
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

403


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [363]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": "leightonconnor/akashic-titan-blue-bolt"
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'fvjxrhwSYir6BG4U761oTRMXCYu4Ql88J4gTJaaXswCLm-qP6vzjo_vw1FdTY3VNpWEmuQb1MZH_JkJ-GuTPew',
  'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
  'cookie': '_ksr_session=XeMoOdlfAuvBg3vZkCPtR2KPkjnWtoE%2FGrwY68dNgE13OYvjkIxO0Eiaf4Oay2xzILiIHuLsmpCAH%2B9aBGT01%2B7gNDH6exw0dtPhWF5xVSYtZnJawqJNc8IUuF52GC%2FS7QHp1woosgy6PABnOt0X0ITzAvUEg8wwMmv5e1IBYNCmj8ZiCiUwlU2h08X3WX0lQSBSeatHjTEHV%2FgAjhqitXq1xvrNIY7MvvonoTMQat4mSbJKrOzGi74PUMwlhlgHz1oDCWy%2BqAiY%2BIQp9ft4c%2BK2ta8%3D--Rds4wbR2FagzmGoy--ZV4JbIYN8J%2FdS8uWWa5EQw%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

429


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [255]:
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
          }
# GET request to kickstarter to get some token needed for later 
r = s.get("https://www.kickstarter.com", headers=headers)
#BeautifulSoup parses the html returned as a response from the GET request 
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

In [ ]:
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'

In [237]:
print("hello")
time.sleep(3)
print("goodnight")

hello
goodnight
